# More Examples of OOP

* Following blocks are one possible implementation of vectors of `double`s.

* Here, member variable `new_name` is in `protected:` part.
* Member methods and subclass members can access this variable but from the outside of the class, we cannot access it.
* We call it **encapsulation**; instead of directly reading or writing to the variable, we would use mutator or reader **methods**.
* This is because to modularize software components to the level of integrated circuit chips.

* Until 1990's, [*software crisis*](https://en.wikipedia.org/wiki/Software_crisis) was one of important issues surrounding the so-called *IT communities* around the world.
* It referred that developing software within time and budget was not easy.
* In my humble opinion, it is still not resolved yet.  However one of the motivations of **object oritented programming** was to help software development more managable by dividing the bigger and more compliex problems into smaller and simpler problems.

``` C++
// Begin vector_double.h

#include <cassert>
#include <cstdint>
#include <exception>
#include <iostream>
#include <string>
#include <vector>


#ifndef LOG
#define LOG
#endif


class RowVector 
{
    // automatic allocation
    // https://stackoverflow.com/questions/8553464/vector-as-a-class-member
    std::vector<double> columns;

    protected:
        std::string name;

    public:
        // Default constructor
		RowVector();

        ~ RowVector();

        // Default arguments
        // If the function could not find the argument in the call, it uses the default value.
        RowVector(const uint32_t n, const double *values=NULL, std::string new_name="None");

        // Instead of implementing another constructor, reusing an existing one
        // c++ 11 or later
        RowVector(const uint32_t n, std::string new_name="None");

        RowVector(const RowVector & other);

        double & operator [] (const uint32_t i);

        const std::string get_name();

        RowVector operator + (const RowVector & other);

        RowVector operator * (const double a);

        const double operator * (const RowVector & other);

        void show();
};

// End vector_double.h

```

``` C++
// Begin vector_double.cpp

#include <cassert>
#include <cstdint>
#include <exception>
#include <iostream>
#include <string>
#include <vector>

#include    "vector_double.h"

RowVector::RowVector(){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "RowVector()" << '\n';
#endif
    name = "None";
}


RowVector::~ RowVector(){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "~ RowVector()" << '\n';
#endif
}


RowVector::RowVector(const uint32_t n, const double *values, std::string new_name){
#ifdef LOG
    std::cout << '[' << &columns << ']' 
    << "RowVector(" << n << ", " << values << ", " << new_name << ")\n";
#endif
    columns.resize(n);

    // If initial values available, copy
    if (values){
        for (uint32_t i = 0; columns.size() > i; ++i){
            columns[i] = values[i];
        }
    }
    // If no initial values, set all values zero
    else{
        for (uint32_t i = 0; columns.size() > i; ++i){
            columns[i] = 0.0;
        }
    }

    name = new_name;
}


RowVector::RowVector(const uint32_t n, std::string new_name) : RowVector(n, NULL, new_name){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "RowVector(" << n << ", " << new_name << ")\n";
#endif
}


RowVector::RowVector(const RowVector & other){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "RowVector(" << & other << ")\n";
#endif
    // https://codereview.stackexchange.com/questions/149669/c-operator-overloading-for-matrix-operations-follow-up
    // http://www.cplusplus.com/reference/vector/vector/resize/
    columns.resize(other.columns.size());
    for(uint32_t i=0; columns.size() > i; ++i){
        columns[i] = other.columns[i];
    }

    // Copy name of the other one
    name = other.name;
    // Then append
    name.append("2");
}


double & RowVector::operator [] (const uint32_t i){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "double & operator [] (" << i << ")\n";
#endif
    // Return reference; otherwise, unable to assign
    return columns[i];
}


const std::string RowVector::get_name(){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "void show()\n";
#endif
    // Return constant; to prevent change
    return name;
}


RowVector RowVector::operator + (const RowVector & other){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "RowVector operator + (" << & other << ")\n";
#endif
    // Check size
    assert(columns.size() == other.columns.size());

    // Make a new vector to return
    RowVector temp(other);

    // Element loop
    for (uint32_t i=0; columns.size() > i; ++i){
        temp[i] += columns[i];
    }

    // Returning a temporary image
    return temp;
}


RowVector RowVector::operator * (const double a){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "RowVector operator * (" << a << ")\n";
#endif

    // Make a new vector to return
    RowVector temp(*this);

    // Element loop in `for each` style
    // c++ 11 or later
    for (auto & element : temp.columns){
        element *= a;
    }

    // Returning a temporary image
    return temp;
}


const double RowVector::operator * (const RowVector & other){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "RowVector operator * (" << & other << ")\n";
#endif

    // Check size
    assert(columns.size() == other.columns.size());

    double dot_product = 0.0;

    // Element loop
    for (uint32_t i = 0; columns.size() > i; ++i){
        dot_product += columns[i] * other.columns[i];
    }

    // Returning a temporary image
    return dot_product;
}


void RowVector::show(){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "void show()\n";
#endif
    for (uint32_t i=0; columns.size()> i; ++i){
        std::cout << name << '[' << i << "] = " << columns[i] << '\n';
    }
}

// End vector_double.cpp
// Build command : g++ -Wall -g -std=c++14 vector_double.cpp -fsyntax-only

```

``` C++
// Begin cpp_vector_double_practice.cpp

#include <cassert>
#include <cstdint>
#include <exception>
#include <iostream>
#include <string>
#include <vector>

#include    "vector_double.h"

int32_t main(int32_t argn, char *argv[]){
	double s[] = {1.0, 2.0};

    std::cout << "RowVector row (2u, s, \"row\");\n";
	RowVector row (2u, s, "row");

    row.show();

    std::cout << "RowVector another_row (row);\n";
	RowVector another_row (row);
    row.show();
    another_row.show();

    std::cout << "another_row[1] += 0.5;\n";
    another_row[1] += 0.5;
    row.show();
    another_row.show();

    std::cout << "RowVector row_plus_another(row + another_row);\n";
    RowVector row_plus_another(row + another_row);
    row.show();
    another_row.show();
    row_plus_another.show();

    std::cout << "RowVector zeros(3);\n";
	RowVector zeros(3u, "zeros");
    row.show();
    another_row.show();
    row_plus_another.show();
    zeros.show();

    double t[] = {2.0, -1.0};
	RowVector ortho (2u, t, "ortho");
    double dot = row * ortho;
    std::cout << "double dot = row * ortho;\n";
    std::cout << "dot  = " << dot << '\n';

    std::cout << "dot = row * row;\n";
    dot = row * row;
    std::cout << "dot  = " << dot << '\n';

}

// End cpp_vector_double_practice.cpp
// Build command : g++ -Wall -g -std=c++14 cpp_vector_double_practice.cpp vector_double.cpp -o cpp_vector_double_practice

``` 

* In the mean while, following code blocks depict a possible implementation in python.

In [ ]:
import collections


class Vector(collections.UserList):

    def __add__(self, other):
        assert len(self) == len(other), f"Lengths are different ({len(self)} == {len(other)})"
        
        # trying list comprehension
        return Vector([a + b for a, b in zip(self, other)])

    def __radd__(self, other):

        # What is this?
        return self.__add__(other)

    def __mul__(self, other):

        if isinstance(other, (int, float, complex)):
            result = Vector([a * other for a in self])
        elif isinstance(other, Vector):
            assert len(self) == len(other),  f"Lengths are different ({len(self)} == {len(other)})"
            result = sum(a * b for a, b in zip(self, other))
        
        return result

    def __rmul__(self, other):
        return __mul__(self, other)
    
    def __str__(self):
        
        return '\n'.join(f"{hex(id(self))}[{i}] = {self[i]}" for i in range(len(self)))



In [ ]:
print("a = Vector([1, 2])")
a = Vector([1, 2])
print(a)

print("b = Vector(a)")
b = Vector(a)
print(a)
print(b)

print("b[1] += (-0.5)")
b[1] += (-0.5)
print(a)
print(b)

print("c = a + b")
c = a + b
print(a)
print(b)
print(c)

print("ortho = Vector([2, -1])")
ortho = Vector([2, -1])
print(a)
print(b)
print(c)
print(ortho)

print("dot = a * ortho")
dot = a * ortho
print(f"a * ortho = {dot}")

print("dot = a * a")
dot = a * a
print(f"a * a = {dot}")

